In [1]:
import requests
from requests import get
import pandas as pd
pd.set_option('display.max_row',None)
pd.set_option('display.max_column',None)

- 사이트 : stocknewsapi.com
- 아이디 : 0603paul@gmail.com
- 비번 : bitamin1!
- api 토큰 : 아래 key 변수

In [3]:
ticker = 'NFLX' # netflix stock
items = 1
pages = [1,2,3]
types = ['article','video']
key = '6tbxlwdn3exj5eugyahnt9clnh9lsp7f82zruvrc'
sortby = 'oldestfirst'
days = list(range(2226, 34,-1)) # 2월4일 기준 2018/01/01은 2225일 전, 23/12/31은 35일 전.

url = f'https://stocknewsapi.com/api/v1'

In [3]:
# try:
#     res = get(url)
# except Exception as e:
#     print(e)

In [25]:
import json
from tqdm import tqdm
from datetime import datetime,timedelta
'''
for1: days는 2018.01.01부터 2023.12.31까지
    for2: page
        for3: type
            1페이지에서 article 1개, video 1개
            2페이지에서 article 1개, video 1개
            3페이지에서 article 1개, video 1개
        
'''
# initialize variables
news_urls_, titles_, texts_, source_names_, dates_, topics_, sentiments_, types_ = [],[],[],[],[],[],[],[]

for day in tqdm(days):
    curr_date = datetime.now()
    print('date :', curr_date - timedelta(days=day))
    for t in types:
        req_params={
            'tickers-only':ticker,
            'sortby':sortby,
            'day':day,
            'items':1,
            'page':1,
            'type':t,
            'token':key
        }
        res = get(url=url, params=req_params)
        res_data = json.loads(res.content)['data'][0]
        news_urls_.append(res_data['news_url'])
        titles_.append(res_data['title'])
        texts_.append(res_data['text'])
        source_names_.append(res_data['source_name'])
        dates_.append(res_data['date'])
        topics_.append(res_data['topics'])
        sentiments_.append(res_data['sentiment'])
        types_.append(res_data['type'])

IndentationError: unexpected indent (588079646.py, line 19)

In [10]:
res_df = pd.DataFrame(data=[news_urls_,titles_,texts_,source_names_,
                  dates_,topics_,sentiments_,types_]).transpose()
res_df.columns=['news_url','title','text','source_name',
               'date','topic','sentiment','news_type']
res_df.head(5)

,news_url,title,text,source_name,date,topic,sentiment,news_type
0,https://www.investopedia.com/articles/insights...,The Top 3 Netflix Shareholders (NFLX),Discover the top owners of Netflix (NFLX) shar...,Investopedia,"Sun, 24 Feb 2019 19:43:11 -0500",[],Positive,Article
1,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,"Tue, 13 Feb 2018 19:00:00 -0500",[],Neutral,Video
2,https://feeds.marketwatch.com/~r/marketwatch/m...,Netflix shares fall premarket after Buckingham...,Netflix Inc. shares fell 1.6% in premarket tra...,Market Watch,"Fri, 08 Mar 2019 09:02:16 -0500",[],Negative,Article
3,https://www.youtube.com/watch?v=pvw45y7DBrM,Netflix and Kill ?,"Robert Thompson, Bleier Center for Television ...",CNBC,"Thu, 29 Mar 2018 20:00:00 -0400",[],Neutral,Video
4,https://feedproxy.google.com/~r/BusinessRss/~3...,"Netflix Stock Downgraded As Disney, Others Pre...",Netflix stock fell Friday after a Wall Street ...,Investors Business Daily,"Fri, 08 Mar 2019 11:12:11 -0500",[],Negative,Article


In [11]:
# res_df.to_csv('../../data/scraping/stocknewsapi.csv',index=0)

# stocknewsapi.csv 파일 정리하기

In [14]:
stocknewsapi_df = pd.read_csv('../../data/scraping/stocknewsapi.csv')

- %a represents the abbreviated weekday name (e.g., 'Thu').
- %d represents the day of the month as zero-padded (01, 02, ..., 31).
- %b represents the abbreviated month name (e.g., 'Mar').
- %Y represents the four-digit year.
- %H represents the hour (00, 01, ..., 23).
- %M represents the minute (00, 01, ..., 59).
- %S represents the second (00, 01, ..., 59).
- %z represents the UTC offset in the form +HHMM or -HHMM.

In [18]:
# 날짜 형식 맞추기
stocknewsapi_df['datetime'] = pd.to_datetime(stocknewsapi_df['date'],format='%a, %d %b %Y %H:%M:%S %z',utc=True)
stocknewsapi_df = stocknewsapi_df.rename(columns={'date':'date_string'})
stocknewsapi_df.head(3)

,news_url,title,text,source_name,date_string,topic,sentiment,news_type,datetime
0,https://www.investopedia.com/articles/insights...,The Top 3 Netflix Shareholders (NFLX),Discover the top owners of Netflix (NFLX) shar...,Investopedia,"Sun, 24 Feb 2019 19:43:11 -0500",[],Positive,Article,2019-02-25 00:43:11+00:00
1,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,"Tue, 13 Feb 2018 19:00:00 -0500",[],Neutral,Video,2018-02-14 00:00:00+00:00
2,https://feeds.marketwatch.com/~r/marketwatch/m...,Netflix shares fall premarket after Buckingham...,Netflix Inc. shares fell 1.6% in premarket tra...,Market Watch,"Fri, 08 Mar 2019 09:02:16 -0500",[],Negative,Article,2019-03-08 14:02:16+00:00


In [19]:
# 날짜 오름차순 정렬
stocknewsapi_df = stocknewsapi_df.sort_values('datetime')
stocknewsapi_df.head(3)

,news_url,title,text,source_name,date_string,topic,sentiment,news_type,datetime
5827,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,"Tue, 13 Feb 2018 19:00:00 -0500",[],Neutral,Video,2018-02-14 00:00:00+00:00
10129,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,"Tue, 13 Feb 2018 19:00:00 -0500",[],Neutral,Video,2018-02-14 00:00:00+00:00
10123,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,"Tue, 13 Feb 2018 19:00:00 -0500",[],Neutral,Video,2018-02-14 00:00:00+00:00


In [22]:
# drop duplicate 
print(stocknewsapi_df.shape)
stocknewsapi_df_dd = stocknewsapi_df.drop_duplicates()
stocknewsapi_df_dd.shape

(13596, 9)


(6, 9)

## 2트

In [49]:
import json
from tqdm import tqdm
from datetime import datetime,timedelta

# initialize variables
news_urls_2, titles_2, texts_2, source_names_2, dates_2, topics_2, sentiments_2, types_2 = [],[],[],[],[],[],[],[]

curr_date = datetime.now()
print('date :', curr_date - timedelta(days=2226))

res_0205 = get(url=url,params={
    'tickers-only':ticker,
    'sortby':sortby,
    'items':1,
    'page':1,
    'token':key
})

total_pages = json.loads(res_0205.content)['total_pages']

for page in tqdm(range(total_pages,0,-1)):
    req_params={
        'tickers-only':ticker,
        'sortby':sortby,
        'day':2226,
        'items':3,
        'page':page,
        'token':key
    }
    res2 = get(url=url, params=req_params)
    res_data2 = json.loads(res.content)['data'][0]
    news_urls_2.append(res_data['news_url'])
    titles_2.append(res_data['title'])
    texts_2.append(res_data['text'])
    source_names_2.append(res_data['source_name'])
    dates_2.append(res_data['date'])
    print(res_data['date'])
    topics_2.append(res_data['topics'])
    sentiments_2.append(res_data['sentiment'])
    types_2.append(res_data['type'])

date : 2018-01-01 13:42:50.331445


  0%|                                                           | 1/5893 [00:01<1:38:38,  1.00s/it]

Sun, 15 Apr 2018 20:00:00 -0400


  0%|                                                           | 2/5893 [00:01<1:26:53,  1.13it/s]

Sun, 15 Apr 2018 20:00:00 -0400


...

Sun, 15 Apr 2018 20:00:00 -0400


  0%|▏                                                         | 14/5893 [00:12<1:26:14,  1.14it/s]

Sun, 15 Apr 2018 20:00:00 -0400


  0%|▏                                                         | 14/5893 [00:13<1:31:02,  1.08it/s]


KeyboardInterrupt: 

In [32]:
res_df = pd.DataFrame(data=[news_urls_2,titles_2,texts_2,source_names_2,
                  dates_2,topics_2,sentiments_2,types_2]).transpose()
res_df.columns=['news_url','title','text','source_name',
               'date','topic','sentiment','news_type']
res_df.head(5)

,news_url,title,text,source_name,date,topic,sentiment,news_type
0,https://www.youtube.com/watch?v=U7suUbVzGH0,"Netflix earnings, subscriber growth beat Wall ...",FBN's Nicole Petallides on how Netflix's earni...,Fox Business,"Sun, 15 Apr 2018 20:00:00 -0400",[],Neutral,Video
1,https://www.youtube.com/watch?v=U7suUbVzGH0,"Netflix earnings, subscriber growth beat Wall ...",FBN's Nicole Petallides on how Netflix's earni...,Fox Business,"Sun, 15 Apr 2018 20:00:00 -0400",[],Neutral,Video
2,https://www.youtube.com/watch?v=U7suUbVzGH0,"Netflix earnings, subscriber growth beat Wall ...",FBN's Nicole Petallides on how Netflix's earni...,Fox Business,"Sun, 15 Apr 2018 20:00:00 -0400",[],Neutral,Video
3,https://www.youtube.com/watch?v=U7suUbVzGH0,"Netflix earnings, subscriber growth beat Wall ...",FBN's Nicole Petallides on how Netflix's earni...,Fox Business,"Sun, 15 Apr 2018 20:00:00 -0400",[],Neutral,Video
4,https://www.youtube.com/watch?v=U7suUbVzGH0,"Netflix earnings, subscriber growth beat Wall ...",FBN's Nicole Petallides on how Netflix's earni...,Fox Business,"Sun, 15 Apr 2018 20:00:00 -0400",[],Neutral,Video


In [33]:
# 날짜 오름차순 정렬
stocknewsapi_df = stocknewsapi_df.sort_values('datetime')
stocknewsapi_df.head(3)

,news_url,title,text,source_name,date_string,topic,sentiment,news_type,datetime
5827,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,"Tue, 13 Feb 2018 19:00:00 -0500",[],Neutral,Video,2018-02-14 00:00:00+00:00
8695,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,"Tue, 13 Feb 2018 19:00:00 -0500",[],Neutral,Video,2018-02-14 00:00:00+00:00
3673,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,"Tue, 13 Feb 2018 19:00:00 -0500",[],Neutral,Video,2018-02-14 00:00:00+00:00


## 3트

In [ ]:
import json
from tqdm import tqdm
from datetime import datetime,timedelta

# initialize variables
news_urls_, titles_, texts_, source_names_, dates_, topics_, sentiments_, types_ = [],[],[],[],[],[],[],[]

curr_date = datetime.now()
print('date :', curr_date - timedelta(days=2226))

res_0205 = get(url=url,params={
    'tickers-only':ticker,
    'sortby':sortby,
    'items':1,
    'page':1,
    'token':key
})

total_pages = json.loads(res_0205.content)['total_pages']

for page in tqdm(range(total_pages,0,-1)):
    req_params={
        'tickers-only':ticker,
        'sortby':sortby,
        'day':2226,
        'items':3,
        'page':page,
        'token':key
    }
    res2 = get(url=url, params=req_params)
    res_data2 = json.loads(res.content)['data'][0]
    news_urls_2.append(res_data['news_url'])
    titles_2.append(res_data['title'])
    texts_2.append(res_data['text'])
    source_names_2.append(res_data['source_name'])
    dates_2.append(res_data['date'])
    print(res_data['date'])
    topics_2.append(res_data['topics'])
    sentiments_2.append(res_data['sentiment'])
    types_2.append(res_data['type'])

# 4트
- 

In [2]:
import json
from tqdm import tqdm
from datetime import datetime,timedelta

In [11]:
curr_date = datetime.now()
print('2018.01.01 date :', curr_date - timedelta(days=2234))
print('2023.12.31 date :', curr_date - timedelta(days=44))
print(list(range(2234,43,-1))[-1])

2018.01.01 date : 2018-01-01 03:29:36.494051
2023.12.31 date : 2023-12-31 03:29:36.494051
44


In [16]:
# empty dataframe
columns = [
    'news_url','title','text','source_name','date','topics','sentiment',
    'type','news_id','rank_score'
]

# initialize variables
ticker = 'NFLX' # netflix stock
items = 100
# pages = [1,2,3,4,5]
# types = ['article','video']
key = '6tbxlwdn3exj5eugyahnt9clnh9lsp7f82zruvrc'
sortby = 'rank'
days = list(range(2234, 44,-1)) # 2월13일 기준 2018/01/01은 2234일 전, 23/12/31은 35일 전.

url = f'https://stocknewsapi.com/api/v1'

for day in tqdm(days):
    req_params={
        'tickers-only':ticker,
        'sortby':sortby,
        'day':day,
        'items':items,
        'page':5,
        'extra-fields':'id,rankscore',
        'token':key
    }
    res_0213_5 = get(url=url, params=req_params)
    print(json.dumps(json.loads(res_0213.content)['data'],indent=4))
    break
    res_data2 = json.loads(res.content)['data'][0]
    news_urls_2.append(res_data['news_url'])
    titles_2.append(res_data['title'])
    texts_2.append(res_data['text'])
    source_names_2.append(res_data['source_name'])
    dates_2.append(res_data['date'])
    print(res_data['date'])
    topics_2.append(res_data['topics'])
    sentiments_2.append(res_data['sentiment'])
    types_2.append(res_data['type'])

  0%|                                                                                         | 0/2190 [00:01<?, ?it/s]

[
    {
        "news_url": "https://www.investors.com/research/netflix-stock-buy-now/",
        "image_url": "https://cdn.snapi.dev/images/v1/l/n/nyf67-2271228.jpg",
        "title": "Is Netflix Stock A Buy After Video Streamer's Fourth-Quarter Report?",
        "text": "Internet television network Netflix has taken a commanding lead in the streaming video market. But is Netflix stock a buy right now?",
        "source_name": "Investors Business Daily",
        "date": "Mon, 12 Feb 2024 07:45:40 -0500",
        "topics": [
            "paywall"
        ],
        "sentiment": "Positive",
        "type": "Article",
        "tickers": [
            "NFLX"
        ],
        "news_id": 2271228,
        "rank_score": "5.04",
        "eventid": null
    },
    {
        "news_url": "https://seekingalpha.com/article/4668602-netflix-long-runway-to-rally-with-peg-below-1x",
        "image_url": "https://cdn.snapi.dev/images/v1/e/l/image-1476749137-2267545.jpg",
        "title": "Netflix: Long

In [15]:
temp = json.loads(res_0213.content)['data']
len(temp)

100

In [23]:
print(json.loads(res_0213.content))
print(json.loads(res_0213_5.content))

{'data': [{'news_url': 'https://www.investors.com/research/netflix-stock-buy-now/', 'image_url': 'https://cdn.snapi.dev/images/v1/l/n/nyf67-2271228.jpg', 'title': "Is Netflix Stock A Buy After Video Streamer's Fourth-Quarter Report?", 'text': 'Internet television network Netflix has taken a commanding lead in the streaming video market. But is Netflix stock a buy right now?', 'source_name': 'Investors Business Daily', 'date': 'Mon, 12 Feb 2024 07:45:40 -0500', 'topics': ['paywall'], 'sentiment': 'Positive', 'type': 'Article', 'tickers': ['NFLX'], 'news_id': 2271228, 'rank_score': '5.04', 'eventid': None}, {'news_url': 'https://seekingalpha.com/article/4668602-netflix-long-runway-to-rally-with-peg-below-1x', 'image_url': 'https://cdn.snapi.dev/images/v1/e/l/image-1476749137-2267545.jpg', 'title': 'Netflix: Long Runway To Rally With PEG Below 1x', 'text': "Netflix's stock has been performing well in 2024, with shares surging over 20% year to date. The company has strong subscriber growth

In [18]:
temp2 = json.loads(res_0213_5.content)['data']
temp2[0]

{'news_url': 'https://www.reuters.com/technology/netflix-may-tap-price-hikes-after-success-password-sharing-crackdown-2023-10-16/',
 'image_url': 'https://cdn.snapi.dev/images/v1/d/4/nmojcwxq7blt3opoqtimtpg4ui-2104094.jpg',
 'title': 'Netflix may tap price hikes after success of password-sharing crackdown',
 'text': "Netflix's crackdown on password-sharing likely boosted subscribers by about 6 million in the third quarter and the streaming pioneer is expected to set the stage for price increases when it reports earnings on Wednesday.",
 'source_name': 'Reuters',
 'date': 'Mon, 16 Oct 2023 06:04:26 -0400',
 'topics': [],
 'sentiment': 'Positive',
 'type': 'Article',
 'tickers': ['NFLX'],
 'news_id': 2104094,
 'rank_score': '6.97',
 'eventid': 'AAAC143'}

In [25]:
df = pd.read_csv('../../data/scraping/stocknewsapi.csv')
df.shape

(13596, 8)

In [32]:
req_params={
    'tickers-only':ticker,
    'sortby':'oldestfirst',
#     'day':day,
    'items':items,
    'page':5,
    'extra-fields':'id,rankscore',
    'token':key
}
res_test = get(url=url, params=req_params)

In [35]:
tmp = json.loads(res_test.content)['data']
len(tmp)

100

# Test - 날짜 지정

In [17]:
req_params={
    'tickers-only':ticker,
#     'sortby':'oldestfirst',
    'date':'01012018-03312018',
    'items':100,
    'page':1,
    'extra-fields':'id,rankscore',
    'token':key
}
url = f'https://stocknewsapi.com/api/v1'
res_test = get(url=url, params=req_params)
print(res_test)

<Response [200]>


In [18]:
import json
print(json.dumps(json.loads(res_test.content)['data'],indent=4))

[
    {
        "news_url": "https://www.youtube.com/watch?v=pvw45y7DBrM",
        "image_url": "https://cdn.snapi.dev/images/v1/p/v/pvw45y7dbrm.jpg",
        "title": "Netflix and Kill ?",
        "text": "Robert Thompson, Bleier Center for Television and Popular Culture at Syracuse University director, and Stephen Battaglio, Los Angeles Times television writer, discuss the landscape for traditional cable and streaming media. They also weigh in on the successful revival of \u201cRoseanne.\u201d",
        "source_name": "CNBC",
        "date": "Thu, 29 Mar 2018 20:00:00 -0400",
        "topics": [],
        "sentiment": "Neutral",
        "type": "Video",
        "tickers": [
            "NFLX"
        ],
        "news_id": 145,
        "rank_score": "6.56"
    },
    {
        "news_url": "https://www.youtube.com/watch?v=vjUW-L35DkA",
        "image_url": "https://cdn.snapi.dev/images/v1/v/j/vjuw-l35dka.jpg",
        "title": "Netflix Lands Big Talent For $300 Million. Why Experts Say

In [19]:
news = json.loads(res_test.content)['data']
len(news)

2

In [25]:
req_params={
    'tickers-only':ticker,
    'date':'10022020-12312020',
    'sortby':'rank',
    'items':100,
    'page':1,
    'extra-fields':'id,rankscore',
    'token':key
}
url = f'https://stocknewsapi.com/api/v1'
res_test2 = get(url=url, params=req_params)
print(res_test2)

<Response [200]>


In [26]:
news2 = json.loads(res_test2.content)['data']
len(news2)

100

In [27]:
print(json.dumps(json.loads(res_test2.content)['data'],indent=4))

[
    {
        "news_url": "https://www.fool.com/investing/2020/12/31/netflix-proved-2-important-points-in-2020/",
        "image_url": "https://cdn.snapi.dev/images/v1/z/g/urlhttps3a2f2fgfoolcdncom2feditorial2fimages2f5262552fnetflix-x1-beta-moviesjpgw700opresize.jpg",
        "title": "Netflix Proved 2 Important Points in 2020",
        "text": "The video-streaming giant is proving its critics wrong in these crucial debates.",
        "source_name": "The Motley Fool",
        "date": "Thu, 31 Dec 2020 08:31:00 -0500",
        "topics": [],
        "sentiment": "Positive",
        "type": "Article",
        "tickers": [
            "NFLX"
        ],
        "news_id": 619513,
        "rank_score": "6.17"
    },
    {
        "news_url": "https://www.fastcompany.com/90590593/netflix-maintains-its-lead-over-streaming-rivals-as-americans-add-disney-hbo-max",
        "image_url": "https://cdn.snapi.dev/images/v1/r/b/poster-streaming-wars-2020-results-data.jpg",
        "title": "Netflix 

In [54]:
ddff = pd.DataFrame(data=d)
ddff['date'] = pd.to_datetime(ddff['date'],format='%a, %d %b %Y %H:%M:%S %z')
ddff.head(5)

C:\Users\songkyu\AppData\Local\Temp\ipykernel_11660\1941828896.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  ddff['date'] = pd.to_datetime(ddff['date'],format='%a, %d %b %Y %H:%M:%S %z',utc=False)


,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers,news_id,rank_score
0,https://www.fool.com/investing/2020/12/31/netf...,https://cdn.snapi.dev/images/v1/z/g/urlhttps3a...,Netflix Proved 2 Important Points in 2020,The video-streaming giant is proving its criti...,The Motley Fool,2020-12-31 08:31:00-05:00,[],Positive,Article,[NFLX],619513,6.17
1,https://www.fastcompany.com/90590593/netflix-m...,https://cdn.snapi.dev/images/v1/r/b/poster-str...,Netflix maintains its lead over streaming riva...,Streaming service subscriptions are up 50% thi...,Fast Company,2020-12-30 15:30:57-05:00,[],Positive,Article,[NFLX],619053,6.30
2,https://www.fool.com/investing/2020/12/30/netf...,https://cdn.snapi.dev/images/v1/e/t/92.jpg,Netflix Stock: Headed to $650?,One analyst thinks these factors could be key ...,The Motley Fool,2020-12-30 08:06:00-05:00,[],Positive,Article,[NFLX],618429,6.17
3,https://www.fool.com/investing/2020/12/30/3-re...,https://cdn.snapi.dev/images/v1/h/w/net6.jpg,3 Reasons I'm Buying Netflix Stock,I believe the streaming giant will keep on win...,The Motley Fool,2020-12-30 07:06:00-05:00,[],Positive,Article,[NFLX],618352,6.17
4,https://pulse2.com/nflx-stock-price-nasdaq-net...,https://cdn.snapi.dev/images/v1/y/x/netflix-3.jpg,NFLX Stock Price: $650 Target By Morgan Stanley,The stock price of Netflix Inc (NASDAQ: NFLX) ...,Pulse2,2020-12-27 08:23:17-05:00,[],Neutral,Article,[NFLX],616062,6.10


In [30]:
d = json.loads(res_test2.content)['data'] 
ddf = pd.DataFrame(data=d)
ddf['date'] = pd.to_datetime(ddf['date'],format='%a, %d %b %Y %H:%M:%S %z',utc=True)
ddf.head(5)

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers,news_id,rank_score
0,https://www.fool.com/investing/2020/12/31/netf...,https://cdn.snapi.dev/images/v1/z/g/urlhttps3a...,Netflix Proved 2 Important Points in 2020,The video-streaming giant is proving its criti...,The Motley Fool,2020-12-31 13:31:00+00:00,[],Positive,Article,[NFLX],619513,6.17
1,https://www.fastcompany.com/90590593/netflix-m...,https://cdn.snapi.dev/images/v1/r/b/poster-str...,Netflix maintains its lead over streaming riva...,Streaming service subscriptions are up 50% thi...,Fast Company,2020-12-30 20:30:57+00:00,[],Positive,Article,[NFLX],619053,6.30
2,https://www.fool.com/investing/2020/12/30/netf...,https://cdn.snapi.dev/images/v1/e/t/92.jpg,Netflix Stock: Headed to $650?,One analyst thinks these factors could be key ...,The Motley Fool,2020-12-30 13:06:00+00:00,[],Positive,Article,[NFLX],618429,6.17
3,https://www.fool.com/investing/2020/12/30/3-re...,https://cdn.snapi.dev/images/v1/h/w/net6.jpg,3 Reasons I'm Buying Netflix Stock,I believe the streaming giant will keep on win...,The Motley Fool,2020-12-30 12:06:00+00:00,[],Positive,Article,[NFLX],618352,6.17
4,https://pulse2.com/nflx-stock-price-nasdaq-net...,https://cdn.snapi.dev/images/v1/y/x/netflix-3.jpg,NFLX Stock Price: $650 Target By Morgan Stanley,The stock price of Netflix Inc (NASDAQ: NFLX) ...,Pulse2,2020-12-27 13:23:17+00:00,[],Neutral,Article,[NFLX],616062,6.10


In [31]:
ddf.sort_values('date')

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers,news_id,rank_score
96,https://invezz.com/news/2020/10/21/netflix-mis...,https://cdn.snapi.dev/images/v1/z/g/netflix2.jpg,Netflix misses expectations for earnings in th...,Netflix Inc. (NASDAQ: NFLX) said on Tuesday th...,Invezz,2020-10-21 10:28:58+00:00,[earnings],Negative,Article,[NFLX],530656,6.78
99,https://www.youtube.com/watch?v=k-pJmpwlINo,https://cdn.snapi.dev/images/v1/m/q/wedbush-ne...,Wedbush: Netflix is a victim of its own success,Joel Kulina of Wedbush Securities discusses wh...,CNBC Television,2020-10-21 11:10:37+00:00,[],Negative,Video,[NFLX],530787,6.55
92,https://www.investopedia.com/netflix-nflx-grow...,https://cdn.snapi.dev/images/v1/0/d/net66.jpg,Netflix (NFLX) Growth Stumbles in the Third Qu...,Netflix (NFLX) missed euphoric third quarter p...,Investopedia,2020-10-21 13:45:16+00:00,[earnings],Negative,Article,[NFLX],531184,7.13
93,https://deadline.com/2020/10/netflix-shares-di...,https://cdn.snapi.dev/images/v1/n/m/nflx-e1563...,"Netflix Shares Dip On Subscriber Slowdown, But...",Netflix shares slipped 6% in early trading tod...,Deadline,2020-10-21 15:00:40+00:00,[earnings],Positive,Article,[NFLX],531310,7.05
91,https://www.youtube.com/watch?v=kRbHioL4Fyg,https://cdn.snapi.dev/images/v1/m/q/here-are-t...,Here are the challenges facing Netflix after t...,"Scott Devitt of Stifel joins ""Squawk on the St...",CNBC Television,2020-10-21 15:49:31+00:00,[earnings],Negative,Video,[NFLX],531440,7.28
...,...,...,...,...,...,...,...,...,...,...,...,...
4,https://pulse2.com/nflx-stock-price-nasdaq-net...,https://cdn.snapi.dev/images/v1/y/x/netflix-3.jpg,NFLX Stock Price: $650 Target By Morgan Stanley,The stock price of Netflix Inc (NASDAQ: NFLX) ...,Pulse2,2020-12-27 13:23:17+00:00,[],Neutral,Article,[NFLX],616062,6.10
3,https://www.fool.com/investing/2020/12/30/3-re...,https://cdn.snapi.dev/images/v1/h/w/net6.jpg,3 Reasons I'm Buying Netflix Stock,I believe the streaming giant will keep on win...,The Motley Fool,2020-12-30 12:06:00+00:00,[],Positive,Article,[NFLX],618352,6.17
2,https://www.fool.com/investing/2020/12/30/netf...,https://cdn.snapi.dev/images/v1/e/t/92.jpg,Netflix Stock: Headed to $650?,One analyst thinks these factors could be key ...,The Motley Fool,2020-12-30 13:06:00+00:00,[],Positive,Article,[NFLX],618429,6.17
1,https://www.fastcompany.com/90590593/netflix-m...,https://cdn.snapi.dev/images/v1/r/b/poster-str...,Netflix maintains its lead over streaming riva...,Streaming service subscriptions are up 50% thi...,Fast Company,2020-12-30 20:30:57+00:00,[],Positive,Article,[NFLX],619053,6.30


# 01012018 ~ 12312023 


In [3]:
from tqdm import tqdm
import warnings
import json
warnings.filterwarnings('ignore')

# empty dataframe
columns = [
    'news_url','title','text','source_name','date','topics','sentiment',
    'type','news_id','rank_score'
]

news_df = pd.DataFrame(data={},columns=columns)

# initialize variables

tickers_only = 'NFLX' # netflix stock
items = 100
pages = 1
# types = ['article','video']
key = '6tbxlwdn3exj5eugyahnt9clnh9lsp7f82zruvrc'
sortby = 'rank'
dates = ['01012018-06312018',
       '07012018-10012018','10022018-12312018',
       '01012019-04012019','04022019-07012019',
       '07022019-10012019','10022019-12312019',
       '01012020-04012020','04022020-07012020',
       '07022020-10012020','10022020-12312020',
        '01012021-02012021','02022021-03012021',
        '03022021-04012021','04022021-05012021',
        '05022021-06012021','06022021-07012021',
        '07022021-08012021','08022021-09012021',
        '09022021-10012021','10022021-11012021',
        '11022021-12012021','12022021-12312021',
        '01012022-02012021','02022021-03012022',
        '03022022-04012022','04022022-05012022',
        '05022022-06012022','06022022-07012022',
        '07022022-08012022','08022022-09012022',
        '09022022-10012022','10022022-11012022',
        '11022022-12012022','12022022-12312022',
        '01012023-02012023','02022023-03012023',
        '03022023-04012023','04022023-05012023',
        '05022023-06012023','06022023-07012023',
        '07022023-08012023','08022023-09012023',
        '09022023-10012023','10022023-11012023',
        '11022023-12012023','12022023-12312023',
       ]
famous_sources = [
    'Bloomberg+Markets+and+Finance',
    'Bloomberg+Technology',
    'Business+Wire',
    'CNBC','CNBC+International+TV','CNBC+Television',
    'CNN','CNN+Business',
    'ETF+Trends','Fast+Company',
    'Yahoo+Finance','Wall+Street+Journel',
    'NYTimes','The+Guardian','Fox+Business',
    'Forbes','New+York+Post','Business+Insider','Reuters'
]
# famous source에 대한 근거
# https://pressgazette.co.uk/media-audience-and-business-data/media_metrics/most-popular-websites-news-world-monthly-2/

url = f'https://stocknewsapi.com/api/v1'

for date in tqdm(dates):
    for source in famous_sources:
        req_params={
            'tickers-only':tickers_only,
            'source':source,
            'sortby':sortby,
            'date':date,
            'items':items,
            'page':pages,
            'extra-fields':'id,rankscore',
            'token':key
        }
        res_0213 = get(url=url, params=req_params)
#         print(json.dumps(json.loads(res_0213.content)['data'],indent=4))
        res_json = json.loads(res_0213.content)['data']
        if len(res_json)==0:
            continue
        res_df = pd.DataFrame(data=res_json).drop(columns=['image_url','tickers'])
        res_df['date'] = pd.to_datetime(res_df['date'],format='%a, %d %b %Y %H:%M:%S %z')
        news_df = pd.concat([news_df,res_df],axis=0)
    

#         news_urls_2.append(res_data['news_url'])
#         titles_2.append(res_data['title'])
#         texts_2.append(res_data['text'])
#         source_names_2.append(res_data['source_name'])
#         dates_2.append(res_data['date'])
#         print(res_data['date'])
#         topics_2.append(res_data['topics'])
#         sentiments_2.append(res_data['sentiment'])
#         types_2.append(res_data['type'])

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [15:14<00:00, 19.47s/it]


In [4]:
### Add data from 2023-12-22 ~ 2023-12-31
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# empty dataframe
columns = [
    'news_url','title','text','source_name','date','topics','sentiment',
    'type','news_id','rank_score'
]

news_df_2 = pd.DataFrame(data={},columns=columns)

# initialize variables

tickers_only = 'NFLX' # netflix stock
items = 60
pages = 1
# types = ['article','video']
key = '6tbxlwdn3exj5eugyahnt9clnh9lsp7f82zruvrc'
sortby = 'rank'
dates = [
        '12222023-12272023','12282023-12312023',
       ]
famous_sources = [
    'Bloomberg+Markets+and+Finance',
    'Bloomberg+Technology',
    'Business+Wire',
    'CNBC','CNBC+International+TV','CNBC+Television',
    'CNN','CNN+Business',
    'ETF+Trends','Fast+Company',
    'Yahoo+Finance','Wall+Street+Journel',
    'NYTimes','The+Guardian','Fox+Business',
    'Forbes','New+York+Post','Business+Insider','Reuters'
]
# famous source에 대한 근거
# https://pressgazette.co.uk/media-audience-and-business-data/media_metrics/most-popular-websites-news-world-monthly-2/

url = f'https://stocknewsapi.com/api/v1'

for date in tqdm(dates):
    for source in famous_sources:
        req_params={
            'tickers-only':tickers_only,
#             'source':source,
            'sortby':sortby,
            'date':date,
            'items':items,
            'page':pages,
            'extra-fields':'id,rankscore',
            'token':key
        }
        res_0213_2 = get(url=url, params=req_params)
#         print(json.dumps(json.loads(res_0213.content)['data'],indent=4))
        res_json_2 = json.loads(res_0213_2.content)['data']
        if len(res_json_2)==0:
            print(date,source)
            continue
        res_df_2 = pd.DataFrame(data=res_json_2).drop(columns=['image_url','tickers'])
        res_df_2['date'] = pd.to_datetime(res_df_2['date'],format='%a, %d %b %Y %H:%M:%S %z')
        news_df_2 = pd.concat([news_df_2,res_df_2],axis=0)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:36<00:00, 18.20s/it]


In [97]:
news_df_2.shape

(190, 10)

In [5]:
news_df_copy = news_df.copy()
news_df_copy['date'] = news_df_copy['date'].apply(lambda x : 
                   pd.to_datetime(str(x)[:11]))
news_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2730 entries, 0 to 1
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   news_url     2730 non-null   object        
 1   title        2730 non-null   object        
 2   text         2730 non-null   object        
 3   source_name  2730 non-null   object        
 4   date         2730 non-null   datetime64[ns]
 5   topics       2730 non-null   object        
 6   sentiment    2730 non-null   object        
 7   type         2730 non-null   object        
 8   news_id      2730 non-null   object        
 9   rank_score   2730 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 234.6+ KB


In [6]:
## There are more data from 12222023 ~ 12292023 excluding famous sources
news_df_2_copy = news_df_2.copy()
news_df_2_copy['date'] = news_df_2_copy['date'].apply(lambda x : 
                   pd.to_datetime(str(x)[:11]))
news_df_sorted_2 = news_df_2_copy.sort_values('date')
news_df_sorted_2 = news_df_sorted_2.set_index('news_id')
display(news_df_sorted_2['date'].iloc[0])
display(news_df_sorted_2['date'].iloc[-1])

Timestamp('2023-12-22 00:00:00')

Timestamp('2023-12-29 00:00:00')

In [7]:
news_df_sorted = news_df_copy.sort_values('date')
news_df_sorted = news_df_sorted.set_index('news_id')

In [8]:
print(news_df_sorted.shape)
display(news_df_sorted.iloc[0])
display(news_df_sorted.iloc[-1])

(2730, 9)


news_url             https://www.youtube.com/watch?v=vjUW-L35DkA
title          Netflix Lands Big Talent For $300 Million. Why...
text           Stephen Battaglio, Los Angeles Times TV and fi...
source_name                                                 CNBC
date                                         2018-02-13 00:00:00
topics                                                        []
sentiment                                                Neutral
type                                                       Video
rank_score                                                  6.56
Name: 353, dtype: object

news_url       https://www.forbes.com/sites/greatspeculations...
title          Will Netflix Stock Rally 40% To Return To Pre-...
text           Netflix stock currently trades at $486 per sha...
source_name                                               Forbes
date                                         2023-12-21 00:00:00
topics                                            [paylimitwall]
sentiment                                               Positive
type                                                     Article
rank_score                                                  5.38
Name: 2204603, dtype: object

In [9]:
news_df = pd.concat([news_df,news_df_2],axis=0)


In [10]:
# news_df.to_csv('../../data/scraping/stocknewsapi_rawdata.csv')

In [72]:
dates_df = pd.DataFrame(date_list,columns=['date'])
(dates_df['date'])

pandas.core.series.Series

In [64]:
date_list = news_df_sorted['date']
criterion = pd.date_range(start='2018-01-01',end='2023-12-31')
criterion_df = pd.DataFrame(data=criterion,columns=['criterion'])
criterion_df.merge

,0
0,2018-01-01
1,2018-01-02
2,2018-01-03
3,2018-01-04
4,2018-01-05
5,2018-01-06
6,2018-01-07
7,2018-01-08
8,2018-01-09
9,2018-01-10


In [100]:
news_df.head(100)

,news_url,title,text,source_name,date,topics,sentiment,type,news_id,rank_score
0,https://www.youtube.com/watch?v=wjJnT3IFn4s,"Cash Burn 'Not a Concern' for Netflix, Says He...","Jason Helfstein, Oppenheimer senior analyst, d...",Bloomberg Technology,2018-04-16 20:00:00-04:00,[],Neutral,Video,417,5.24
1,https://www.youtube.com/watch?v=8NS96g6TlWg,Netflix Tops Revenue and Subscriber Estimates,Netflix first-quarter profit rose to 64 cents ...,Bloomberg Technology,2018-04-15 20:00:00-04:00,[],Neutral,Video,411,5.24
0,https://www.youtube.com/watch?v=pvw45y7DBrM,Netflix and Kill ?,"Robert Thompson, Bleier Center for Television ...",CNBC,2018-03-29 20:00:00-04:00,[],Neutral,Video,145,6.56
1,https://www.youtube.com/watch?v=vjUW-L35DkA,Netflix Lands Big Talent For $300 Million. Why...,"Stephen Battaglio, Los Angeles Times TV and fi...",CNBC,2018-02-13 19:00:00-05:00,[],Neutral,Video,353,6.56
0,https://www.youtube.com/watch?v=PAwREs7snJk,How Netflix is everything to everyone,Netflix is sitting on the streaming throne bec...,CNN,2018-06-28 20:00:00-04:00,[],Neutral,Video,1190,5.61
0,https://www.youtube.com/watch?v=HImeospbbac,Netflix can't be taken over?,SteelHouse CEO Mark Douglas on the future of N...,Fox Business,2018-06-12 20:00:00-04:00,[],Neutral,Video,1085,6.49
1,https://www.youtube.com/watch?v=U7suUbVzGH0,"Netflix earnings, subscriber growth beat Wall ...",FBN's Nicole Petallides on how Netflix's earni...,Fox Business,2018-04-15 20:00:00-04:00,[],Neutral,Video,415,6.49
0,https://www.youtube.com/watch?v=6IdS8LrQAnM,Netflix plans to double its investment in France,CNBC's 'Squawk Box' team discusses Netflix CEO...,CNBC Television,2018-09-27 20:00:00-04:00,[],Neutral,Video,3705,6.54
1,https://www.youtube.com/watch?v=yI-IhtCIq04,Legendary agent Michael Ovitz discusses Netfli...,"Michael Ovitz, co-founder of Creative Artists ...",CNBC Television,2018-09-24 20:00:00-04:00,[],Neutral,Video,3583,6.54
2,https://www.youtube.com/watch?v=-9tiPJBD0_Y,Traders discuss whether Netflix will be bigges...,Is Netflix the new king of content? With CNBC'...,CNBC Television,2018-09-16 20:00:00-04:00,[],Neutral,Video,3160,6.54


In [104]:
test = news_df['topics'].apply(lambda x : None if len(x)==0 else x)


In [105]:
test.isnull().sum()

1758

In [106]:
news_df.shape

(2920, 10)

# 01012018 ~ 12312023 
- source_name 미지정

In [4]:
from requests import get
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import json
warnings.filterwarnings('ignore')

# empty dataframe
columns = [
    'news_url','title','text','source_name','date','topics','sentiment',
    'type','news_id','rank_score'
]

news_df = pd.DataFrame(data={},columns=columns)

# initialize variables

tickers_only = 'NFLX' # netflix stock
items = 100
pages = 1
# types = ['article','video']
key = '6tbxlwdn3exj5eugyahnt9clnh9lsp7f82zruvrc'
sortby = 'rank'
dates = ['01012018-06312018',
       '07012018-10012018','10022018-12312018',
       '01012019-04012019','04022019-07012019',
       '07022019-10012019','10022019-12312019',
       '01012020-04012020','04022020-07012020',
       '07022020-10012020','10022020-12312020',
        '01012021-02012021','02022021-03012021',
        '03022021-04012021','04022021-05012021',
        '05022021-06012021','06022021-07012021',
        '07022021-08012021','08022021-09012021',
        '09022021-10012021','10022021-11012021',
        '11022021-12012021','12022021-12312021',
        '01012022-02012021','02022021-03012022',
        '03022022-04012022','04022022-05012022',
        '05022022-06012022','06022022-07012022',
        '07022022-08012022','08022022-09012022',
        '09022022-10012022','10022022-11012022',
        '11022022-12012022','12022022-12312022',
        '01012023-02012023','02022023-03012023',
        '03022023-04012023','04022023-05012023',
        '05022023-06012023','06022023-07012023',
        '07022023-08012023','08022023-09012023',
        '09022023-10012023','10022023-11012023',
        '11022023-12012023','12022023-12312023',
       ]

url = f'https://stocknewsapi.com/api/v1'

for date in tqdm(dates):
    req_params={
        'tickers-only':tickers_only,
        'sortby':sortby,
        'date':date,
        'items':items,
        'page':pages,
        'extra-fields':'id,rankscore',
        'token':key
    }
    res_0213 = get(url=url, params=req_params)
#         print(json.dumps(json.loads(res_0213.content)['data'],indent=4))
    res_json = json.loads(res_0213.content)['data']
    if len(res_json)==0:
        continue
    res_df = pd.DataFrame(data=res_json).drop(columns=['image_url','tickers'])
    res_df['date'] = pd.to_datetime(res_df['date'],format='%a, %d %b %Y %H:%M:%S %z')
    news_df = pd.concat([news_df,res_df],axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [01:06<00:00,  1.41s/it]


In [6]:
### Add data from 2023-12-22 ~ 2023-12-31
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# empty dataframe
columns = [
    'news_url','title','text','source_name','date','topics','sentiment',
    'type','news_id','rank_score'
]

news_df_2 = pd.DataFrame(data={},columns=columns)

# initialize variables

tickers_only = 'NFLX' # netflix stock
items = 60
pages = 1
# types = ['article','video']
key = '6tbxlwdn3exj5eugyahnt9clnh9lsp7f82zruvrc'
sortby = 'rank'
dates = [
        '12222023-12272023','12282023-12312023',
       ]

# famous source에 대한 근거
# https://pressgazette.co.uk/media-audience-and-business-data/media_metrics/most-popular-websites-news-world-monthly-2/

url = f'https://stocknewsapi.com/api/v1'

for date in tqdm(dates):
    req_params={
        'tickers-only':tickers_only,
        'sortby':sortby,
        'date':date,
        'items':items,
        'page':pages,
        'extra-fields':'id,rankscore',
        'token':key
    }
    res_0213_2 = get(url=url, params=req_params)
#         print(json.dumps(json.loads(res_0213.content)['data'],indent=4))
    res_json_2 = json.loads(res_0213_2.content)['data']
    if len(res_json_2)==0:
        print(date,source)
        continue
    res_df_2 = pd.DataFrame(data=res_json_2).drop(columns=['image_url','tickers'])
    res_df_2['date'] = pd.to_datetime(res_df_2['date'],format='%a, %d %b %Y %H:%M:%S %z')
    news_df_2 = pd.concat([news_df_2,res_df_2],axis=0)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.02it/s]


In [7]:
print(news_df.shape)
news_df_2.shape

(3410, 10)


(10, 10)

In [8]:
news_df_copy = news_df.copy()
news_df_copy['date'] = news_df_copy['date'].apply(lambda x : 
                   pd.to_datetime(str(x)[:11]))
news_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3410 entries, 0 to 48
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   news_url     3410 non-null   object        
 1   title        3410 non-null   object        
 2   text         3410 non-null   object        
 3   source_name  3410 non-null   object        
 4   date         3410 non-null   datetime64[ns]
 5   topics       3410 non-null   object        
 6   sentiment    3410 non-null   object        
 7   type         3410 non-null   object        
 8   news_id      3410 non-null   object        
 9   rank_score   3410 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 293.0+ KB


In [9]:
## There are more data from 12222023 ~ 12292023 excluding famous sources
news_df_2_copy = news_df_2.copy()
news_df_2_copy['date'] = news_df_2_copy['date'].apply(lambda x : 
                   pd.to_datetime(str(x)[:11]))
news_df_sorted_2 = news_df_2_copy.sort_values('date')
news_df_sorted_2 = news_df_sorted_2.set_index('news_id')
display(news_df_sorted_2['date'].iloc[0])
display(news_df_sorted_2['date'].iloc[-1])

Timestamp('2023-12-22 00:00:00')

Timestamp('2023-12-29 00:00:00')

In [10]:
news_df_sorted = news_df_copy.sort_values('date')
news_df_sorted = news_df_sorted.set_index('news_id')

In [11]:
print(news_df_sorted.shape)
display(news_df_sorted.iloc[0])
display(news_df_sorted.iloc[-1])

(3410, 9)


news_url             https://www.youtube.com/watch?v=vjUW-L35DkA
title          Netflix Lands Big Talent For $300 Million. Why...
text           Stephen Battaglio, Los Angeles Times TV and fi...
source_name                                                 CNBC
date                                         2018-02-13 00:00:00
topics                                                        []
sentiment                                                Neutral
type                                                       Video
rank_score                                                  6.56
Name: 353, dtype: object

news_url       https://www.zacks.com/stock/news/2203890/netfl...
title          Netflix (NFLX) Falls More Steeply Than Broader...
text           Netflix (NFLX) reachead $486.88 at the closing...
source_name                            Zacks Investment Research
date                                         2023-12-29 00:00:00
topics                                                        []
sentiment                                                Neutral
type                                                     Article
rank_score                                                  4.87
Name: 2211934, dtype: object

In [12]:
news_df = pd.concat([news_df,news_df_2],axis=0)


In [13]:
news_df.shape

(3420, 10)

In [14]:
news_df.to_csv('../../data/scraping/stocknewsapi_rawdata_0214.csv')